In [1]:
import os, sys, warnings
import traceback
import sqlite3 as sq
import pandas as pd
from pathlib import Path

PATH_DB = 'mydb.db'
warnings.filterwarnings('ignore')

In [2]:
file_db = Path('./mydb.db')
if not file_db.exists():
    !type nul>mydb.db

In [3]:
input_data =  Path('./input_data/supermarket_sales - Sheet1.csv')
if not input_data.exists():
    !kaggle datasets download -d aungpyaeap/supermarket-sales -p ./input_data/ --unzip


  0%|          | 0.00/35.9k [00:00<?, ?B/s]
100%|##########| 35.9k/35.9k [00:00<00:00, 799kB/s]


In [43]:
colnames=['invoice_ID', 'branch', 'city', 'customer_type', 'gender', 'product_line', 'unit_price', 'quantity', 'tax', 'total', 'date', 'time', 'payment', 'cogs', 'gross_margin_percentage', 'gross_income', 'rating']

In [44]:
df = pd.read_csv('input_data/supermarket_sales - Sheet1.csv')

In [45]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [46]:
df.columns = colnames

In [47]:
df.head()

,invoice_ID,branch,city,customer_type,gender,product_line,unit_price,quantity,tax,total,date,time,payment,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [59]:
df.customer_type.value_counts()

customer_type
Member    501
Normal    499
Name: count, dtype: int64

**Необходимые функции**

In [48]:
def create_connecting(db_file):
    """Connect to DB"""
    try:
        conn = sq.connect(db_file)
    except sq.Error as db_err:
        print(f'DBError: {db_err}')
    return conn


def open_file(path):
    """Open file"""
    with open(path, 'r') as f:
        file = f.read()
    return file


def read_sql(conn_path, sql_path):
    """Execute SQL-script"""
    conn = create_connecting(conn_path)
    try:
        curr = conn.cursor()
        sql = open_file(sql_path)
        curr.executescript(sql)
        conn.commit()
        print(f'Script done')
    except IOError as io_err:
        print(f'IOError: {io_err}')
    except sq.Error as db_err:
        print(f'DBError: {db_err}')
    finally:
        conn.close()
        
def update_data(conn_path, data, table):
    """Update data in table"""
    data = data.astype('str')
    conn = create_connecting(conn_path)
    
    q = f"""INSERT INTO {table}({",".join(list(data))})
            VALUES ({",".join(['?' for x in range(data.shape[1])])})"""
    
    try:
        curr = conn.cursor()
        curr.executemany(q, list(data.to_records(index=False)))
        conn.commit()
        print(f'Data update is Success')
    except IOError as io_err:
        print(f'IOError: {io_err}')
    except sq.Error as db_err:
        print(f'DBError: {db_err}')
        print('SQLite traceback: ')
        exc_type, exc_value, exc_tb = sys.exc_info()
        print(traceback.format_exception(exc_type, exc_value, exc_tb))
    finally:
        conn.close() 

In [52]:
supermarket_sales = """
DROP TABLE if exists STG_SUPERMARKET_SALES;
CREATE TABLE if not exists STG_SUPERMARKET_SALES(
invoice_ID varchar(128),
branch varchar(128),
city varchar(128),
customer_type varchar(128),
gender varchar(128),
product_line varchar(128),
unit_price varchar(128),
quantity varchar(128),
tax varchar(128),
total varchar(128),
date varchar(128),
time varchar(128),
payment varchar(128),
cogs varchar(128),
gross_margin_percentage varchar(128),
gross_income varchar(128),
rating varchar(128));
"""

In [50]:
files = os.listdir('sql_scripts/')
files = [file for file in files if 'create' in file]

In [13]:
files

['create_STG_SUPERMARKET_SALES.sql']

In [54]:
for file in files:
    read_sql(PATH_DB, f'sql_scripts/{file}')

Script done


**Проверим, создались ли таблицы**

Таблицы создались успешно

In [55]:
update_data(PATH_DB, df, 'STG_SUPERMARKET_SALES')

Data update is Success


In [56]:
pd.read_sql_query("""
SELECT *
FROM STG_SUPERMARKET_SALES
""", con = create_connecting(PATH_DB))

,invoice_ID,branch,city,customer_type,gender,product_line,unit_price,quantity,tax,total,date,time,payment,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761904762,2.0175,6.2
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.69,1022.49,3/2/2019,17:16,Ewallet,973.8,4.761904762,48.69,4.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.592,33.432,2/9/2019,13:22,Cash,31.84,4.761904762,1.592,7.7
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.291,69.111,2/22/2019,15:33,Cash,65.82,4.761904762,3.291,4.1
